In [ ]:
!pip uninstall -y torchao

!apt-get update
!apt-get install -y ffmpeg libavcodec-dev libavformat-dev libavdevice-dev libavfilter-dev libavutil-dev libswscale-dev libswresample-dev

!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft

!pip install librosa torchaudio

In [ ]:
import os

print("🔍 Ψάχνω για αρχεία ήχου στο /kaggle/input...")

found_files = []
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.endswith((".wav", ".mp3")):
            full_path = os.path.join(root, file)
            found_files.append(full_path)
            if len(found_files) <= 3:
                print(f"✅ Βρέθηκε: {full_path}")

if not found_files:
    print("❌ Δεν βρέθηκε κανένα αρχείο ήχου! Βεβαιώσου ότι ανέβασες το Dataset.")
else:
    print(f"\nΣυνολικά βρέθηκαν {len(found_files)} αρχεία.")
    suggested_dir = os.path.dirname(found_files[0])
    print(f"\n💡 Ο σωστός φάκελος (AUDIO_DIR) μάλλον είναι:\n'{suggested_dir}'")

🔍 Ψάχνω για αρχεία ήχου στο /kaggle/input...
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_41/original.wav
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_41/suspicious.wav
✅ Βρέθηκε: /kaggle/input/thesis-audio-files/pair_17/original.wav

Συνολικά βρέθηκαν 100 αρχεία.

💡 Ο σωστός φάκελος (AUDIO_DIR) μάλλον είναι:
'/kaggle/input/thesis-audio-files/pair_41'


In [ ]:
import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
import os
import random
from tqdm import tqdm
import glob

BASE_INPUT_DIR = "/kaggle/input/thesis-audio-files"
OUTPUT_DIR = "/kaggle/working/generated_covers"
os.makedirs(OUTPUT_DIR, exist_ok=True)

NUM_COVERS = 30
DURATION = 30 # seconds

# Music Style for Hard Positives
STYLES = [
    "heavy metal rock song, energetic, distorted guitar",
    "smooth jazz piano, relaxing, lounge atmosphere",
    "electronic dance music, techno, synth bass, 140 bpm",
    "classical orchestral symphony, violins, dramatic",
    "reggae song, slow beat, jamaican style",
    "acoustic guitar folk song, campfire vibe",
    "8-bit chiptune video game music, retro",
    "hip hop beat, old school, boom bap drums"
]

print("🔍 Scanning for original.wav files...")
all_originals = glob.glob(os.path.join(BASE_INPUT_DIR, "**", "original.wav"), recursive=True)

# If there is no original.wav, we search for all .wav
if not all_originals:
    all_originals = glob.glob(os.path.join(BASE_INPUT_DIR, "**", "*.wav"), recursive=True)

print(f"✅ Found {len(all_originals)} source files.")

if len(all_originals) == 0:
    raise ValueError("Δεν βρέθηκαν αρχεία! Έλεγξε το path.")

# Select random file
if len(all_originals) > NUM_COVERS:
    selected_files = random.sample(all_originals, NUM_COVERS)
else:
    selected_files = all_originals

# Load Model
print("⏳ Loading MusicGen...")
model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=DURATION)

# Generation Loop
print(f"🚀 Generating {len(selected_files)} AI Covers...")

for idx, file_path in enumerate(tqdm(selected_files)):
    try:
        # Load Original (Melody)
        melody, sr = torchaudio.load(file_path)

        # If song shorter than 30 secs, ignore or cut it
        if melody.shape[1] / sr < 5.0: # Skip very short files
            continue

        # Select random style
        style = random.choice(STYLES)
        description = [style]

        # Generation
        wav = model.generate_with_chroma(
            descriptions=description,
            melody_wavs=melody[None],
            melody_sample_rate=sr
        )

        # Save
        wav = wav.detach().cpu().squeeze(0)
        # Naming e.g. cover_pair1_jazz
        parent_folder = os.path.basename(os.path.dirname(file_path))
        safe_style = style.split(',')[0].replace(' ', '_')
        save_name = f"cover_{parent_folder}_{safe_style}_{idx}"

        save_path = os.path.join(OUTPUT_DIR, save_name)
        audio_write(save_path, wav, model.sample_rate, strategy="loudness", loudness_headroom_db=16)

    except Exception as e:
        print(f"❌ Error on {file_path}: {e}")
        continue

print(f"✅ Έτοιμα! Τα covers είναι στο {OUTPUT_DIR}")

🔍 Scanning for original.wav files...
✅ Found 51 source files.
⏳ Loading MusicGen...


state_dict.bin:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 143MB/s]


compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


🚀 Generating 30 AI Covers...


100%|██████████| 30/30 [50:41<00:00, 101.39s/it]

✅ Έτοιμα! Τα covers είναι στο /kaggle/working/generated_covers


In [ ]:
import shutil
import os

SOURCE_DIR = "/kaggle/working/generated_covers"
OUTPUT_FILENAME = "/kaggle/working/ai_covers_audio"

print(f"📦 Zipping files from {SOURCE_DIR}...")

if os.path.exists(SOURCE_DIR) and len(os.listdir(SOURCE_DIR)) > 0:
    # ZIP
    shutil.make_archive(OUTPUT_FILENAME, 'zip', SOURCE_DIR)

    print(f"\n✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: {OUTPUT_FILENAME}.zip")
    print("👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio.zip'.")
else:
    print("❌ Σφάλμα: Ο φάκελος είναι άδειος ή δεν υπάρχει.")

📦 Zipping files from /kaggle/working/generated_covers...

✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: /kaggle/working/ai_covers_audio.zip
👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio.zip'.


In [ ]:
from IPython.display import FileLink

FileLink('ai_covers_audio.zip')

/kaggle/working/ai_covers_audio.zip

In [ ]:
# MERT dependencies
!pip install transformers torchaudio

In [ ]:
!pip install transformers==4.46.3

!pip install torchaudio librosa numpy

In [ ]:
import shutil
import os

SOURCE_DIR = "/kaggle/working/ai_covers_segmented"
OUTPUT_FILENAME = "/kaggle/working/ai_covers_audio_segments"

print(f"📦 Zipping files from {SOURCE_DIR}...")

if os.path.exists(SOURCE_DIR) and len(os.listdir(SOURCE_DIR)) > 0:
    # ZIP
    shutil.make_archive(OUTPUT_FILENAME, 'zip', SOURCE_DIR)

    print(f"\n✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: {OUTPUT_FILENAME}.zip")
    print("👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio_segments.zip'.")
else:
    print("❌ Σφάλμα: Ο φάκελος είναι άδειος ή δεν υπάρχει.")

from IPython.display import FileLink

FileLink('ai_covers_audio_segments.zip')

📦 Zipping files from /kaggle/working/ai_covers_segmented...

✅ ΕΤΟΙΜΟ! Το αρχείο δημιουργήθηκε: /kaggle/working/ai_covers_audio_segments.zip
👉 Πήγαινε δεξιά στο μενού 'Output', κάνε refresh αν χρειαστεί, και κατέβασε το 'ai_covers_audio_segments.zip'.


/kaggle/working/ai_covers_audio_segments.zip

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

INPUT_DIR = "/kaggle/working/generated_covers"      # path to wavs created by MusicGen
OUTPUT_DIR = "/kaggle/working/ai_covers_segmented"  # segments
BEATS_PER_SEGMENT = 16

try:
    import librosa
except ImportError:
    os.system('pip install librosa soundfile')
    import librosa

def segment_track_beat_synced(file_path, output_folder):
    try:
        # Load
        y, sr = librosa.load(file_path, sr=None)

        # Beat Tracking
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        beat_samples = librosa.frames_to_samples(beat_frames)

        # If not enough beats, ignore it
        if len(beat_samples) < BEATS_PER_SEGMENT:
            print(f"⚠️ Warning: Λίγα beats στο {os.path.basename(file_path)}. Skipping.")
            return

        # Segmentation Loop
        num_segments = 0
        for i in range(0, len(beat_samples) - BEATS_PER_SEGMENT, BEATS_PER_SEGMENT):
            start_sample = beat_samples[i]
            end_sample = beat_samples[i + BEATS_PER_SEGMENT]

            segment = y[start_sample:end_sample]

            # ignore small segments (< 2 sec)
            if len(segment) / sr < 2.0:
                continue

            # Save
            seg_filename = f"{num_segments}.wav"
            sf.write(os.path.join(output_folder, seg_filename), segment, sr)
            num_segments += 1

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")

# MAIN EXECUTION
print("🚀 Starting 4-Bar Segmentation on AI Covers...")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Locate wav files
cover_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
print(f"📂 Found {len(cover_files)} covers to process.")

for filename in tqdm(cover_files):
    input_path = os.path.join(INPUT_DIR, filename)

    # Subfile e.g. ai_covers_segmented/cover_pair_1_jazz_0/
    song_name = os.path.splitext(filename)[0]
    song_output_dir = os.path.join(OUTPUT_DIR, song_name)
    os.makedirs(song_output_dir, exist_ok=True)

    # Execute Segmentation
    segment_track_beat_synced(input_path, song_output_dir)

print(f"\n✅ Segmentation Complete! Files are in {OUTPUT_DIR}")

🚀 Starting 4-Bar Segmentation on AI Covers...
📂 Found 30 covers to process.


100%|██████████| 30/30 [00:17<00:00,  1.70it/s]


✅ Segmentation Complete! Files are in /kaggle/working/ai_covers_segmented


In [ ]:
import torch
import os
import numpy as np
from transformers import AutoModel, Wav2Vec2FeatureExtractor
import torchaudio
from tqdm import tqdm
import glob

INPUT_SEGMENTED_DIR = "/kaggle/working/ai_covers_segmented"

OUTPUT_EMB_DIR = "/kaggle/working/hard_positive_embeddings"
os.makedirs(OUTPUT_EMB_DIR, exist_ok=True)

def run_mert_model_and_get_features(waveforms, audio_model, time_reduce=None):
    if time_reduce is None:
        time_reduce = torch.nn.AvgPool1d(kernel_size=10, stride=10, count_include_pad=False)

    outputs = audio_model(waveforms, output_hidden_states=True)
    hidden_states = outputs.hidden_states

    # Selection of specific layers [2, 5, 8, 11] -> 4 Layers total
    audio_features = torch.stack(
        [time_reduce(h.detach()[:, :, :].permute(0,2,1)).permute(0,2,1) for h in hidden_states[2::3]], dim=1
    )
    return audio_features


def extract_hard_positives_structured():
    print("🚀 Starting Structured MERT Extraction (Robust Save)...")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    processor = Wav2Vec2FeatureExtractor.from_pretrained("m-a-p/MERT-v1-95M")
    model = AutoModel.from_pretrained("m-a-p/MERT-v1-95M", trust_remote_code=True).to(device)
    model.eval()

    time_reduce = torch.nn.AvgPool1d(kernel_size=10, stride=10, count_include_pad=False).to(device)

    song_folders = [f for f in os.listdir(INPUT_SEGMENTED_DIR) if os.path.isdir(os.path.join(INPUT_SEGMENTED_DIR, f))]
    print(f"📂 Found {len(song_folders)} cover songs (folders).")

    for song_folder in tqdm(song_folders):
        song_path = os.path.join(INPUT_SEGMENTED_DIR, song_folder)

        # Sort
        segment_files = sorted(
            [f for f in os.listdir(song_path) if f.endswith('.wav')],
            key=lambda x: int(os.path.splitext(x)[0])
        )

        if not segment_files:
            continue

        all_segments_features = []

        for seg_file in segment_files:
            seg_path = os.path.join(song_path, seg_file)

            # Loading
            waveform, sr = torchaudio.load(seg_path)
            if sr != 24000:
                resampler = torchaudio.transforms.Resample(sr, 24000)
                waveform = resampler(waveform)

            # Extraction
            inputs = processor(waveform.squeeze(), sampling_rate=24000, return_tensors="pt", padding=True)
            input_values = inputs.input_values.to(device)

            with torch.no_grad():
                features = run_mert_model_and_get_features(input_values, model, time_reduce)

            # (4, Time, 768)
            all_segments_features.append(features.cpu().squeeze(0).numpy())

        ragged_array = np.empty(len(all_segments_features), dtype=object)

        for i, feat in enumerate(all_segments_features):
            ragged_array[i] = feat

        save_name = f"{song_folder}.npy"
        np.save(os.path.join(OUTPUT_EMB_DIR, save_name), ragged_array)

    print(f"✅ Extraction Complete! Embeddings saved in {OUTPUT_EMB_DIR}")

extract_hard_positives_structured()

🚀 Starting Structured MERT Extraction (Robust Save)...
Using device: cuda
📂 Found 30 cover songs (folders).


100%|██████████| 30/30 [00:04<00:00,  6.03it/s]

✅ Extraction Complete! Embeddings saved in /kaggle/working/hard_positive_embeddings


In [ ]:
import shutil
shutil.make_archive("hard_positives", 'zip', "/kaggle/working/hard_positive_embeddings")

'/kaggle/working/hard_positives.zip'

In [ ]:
FileLink('hard_positives.zip')

/kaggle/working/hard_positives.zip